<a href="https://colab.research.google.com/github/kokorikos212/Odysseus_Elytis_model/blob/main/ELytis_finetune2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset

from datasets import Dataset

import pandas as pd

In [3]:
import os
from transformers import AutoTokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
folder_path = "/content/drive/MyDrive/poetry_segments"
# Read all text files from the folder
texts = []
for filename in os.listdir(folder_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            texts.append(file.read())

# Create a DataFrame with the texts
df = pd.DataFrame({'text': texts})
dataset = Dataset.from_pandas(df)
print(f"Number of samples: {len(dataset)}")

Number of samples: 490


In [6]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Set the pad_token to be the eos_token
tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=1024)

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=['text'])


Map:   0%|          | 0/490 [00:00<?, ? examples/s]

In [8]:
# Split the dataset
train_test_split = tokenized_dataset.train_test_split(test_size=0.1)  # Adjust test_size as needed

train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

In [9]:
train_dataset.column_names

['input_ids', 'attention_mask']

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForCausalLM

# Load the model
model = AutoModelForCausalLM.from_pretrained('gpt2')

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [11]:
training_args = TrainingArguments(
    output_dir="./elytis_gpt2",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=True,
    remove_unused_columns=False
)
print(tokenized_dataset[0])


{'input_ids': [17394, 46582, 139, 234, 38392, 32830, 17394, 38392, 138, 115, 18919, 138, 115, 26180, 139, 236, 26180, 26367, 43000, 139, 234, 34703, 138, 115, 220, 26367, 39377, 39377, 138, 105, 7377, 112, 29945, 17394, 43000, 33643, 138, 107, 26180, 26517, 26180, 32830, 17394, 29945, 7377, 106, 138, 112, 138, 115, 46651, 17394, 18074, 223, 26517, 139, 227, 138, 116, 26517, 139, 235, 26180, 29945, 17394, 7377, 118, 139, 234, 43000, 43000, 29945, 26180, 17394, 18074, 222, 26517, 139, 227, 18074, 222, 26517, 39377, 139, 235, 7377, 116, 138, 255, 39377, 26517, 139, 227, 26180, 18074, 222, 26517, 32830, 138, 255, 18074, 222, 29945, 17394, 7377, 121, 17394, 18919, 138, 115, 26180, 7377, 113, 138, 107, 38392, 17394, 29945, 7377, 118, 139, 227, 33643, 29945, 17394, 43000, 138, 115, 220, 220, 27169, 18074, 228, 32830, 138, 105, 26180, 26517, 26180, 32830, 17394, 35558, 46651, 26517, 27169, 17394, 46582, 139, 234, 33643, 29945, 18919, 30950, 42063, 138, 105, 39377, 49535, 38392, 30950, 7377, 11

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# Start fine-tuning
trainer.train()

In [ ]:
from google.colab import files
import shutil

# Compress the directory
shutil.make_archive('/content/model_archive', 'zip', '/content/model_directory')

# Download the zip file
files.download('/content/model_archive.zip')

In [ ]:
# Prepare input text
def preprocess_input(text, tokenizer):
    return tokenizer(text, return_tensors="pt", padding=True, truncation=True)

text = "Καλώς εχό"
inputs = preprocess_input(text, tokenizer)


In [ ]:
def get_predictions(inputs, model):
    with torch.no_grad():
        outputs = model(**inputs)
        # Assuming the model is a classification model
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    return predictions

# Get model predictions
predictions = get_predictions(inputs, model)
print(predictions)



tensor([[  198,   138, 32830,   138,   234,   138,  7377,   118,   157,   234,
         17394,   234, 35558]])


RuntimeError: a Tensor with 13 elements cannot be converted to Scalar

In [ ]:
# Parameters for text generation
max_length = 200  # Max length of the generated text
temperature = 0.7  # Sampling temperature to control randomness
top_k = 50  # Top-K sampling for more diverse generation
top_p = 0.9  # Top-p sampling (nucleus sampling)

In [ ]:
def generate_text(prompt, model, tokenizer, max_length=2000, temperature=0.7, top_k=50, top_p=0.9):
    # Tokenize the input prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text using the model
    output = model.generate(
        input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        no_repeat_ngram_size=2,  # Avoid repeating phrases
        do_sample=True,  # Enable sampling
    )

    # Decode the generated tokens into a human-readable string
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [ ]:
# Example prompt to generate Elytis-like text
prompt = "Έτσι μιλώ για σένα και για μένα"

# Generate and print the text
elytis_like_text = generate_text(prompt, model, tokenizer, max_length, temperature, top_k, top_p)
print(elytis_like_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Έτσι μιλώ για σένα και για μένα τοῦ πρόποτής διηταδυσαχείαν οὐτὰ εὼ λαρτος υπθξασάζοι ἐκΤβΛΡεΣΙσεἱ ηὦτι χγριον αὴ θεὶ Καττρεστες μοοᾶς τẓ Πολλαὸς ὅτων φνομισκύτψα


In [ ]:
def format_as_poetry(text):
    lines = text.split('.')
    formatted_poetry = '\n'.join([line.strip() for line in lines if line.strip()])
    return formatted_poetry

# Format the generated text as poetry
poetic_text = format_as_poetry(elytis_like_text)
print(poetic_text)